In [ ]:
import gwaslab as gl

In [2]:
mysumstats = gl.Sumstats(
    "/mnt/hdd_1/ofgeha/galaxy-gwas-tools/Data/21001_T2T_liftover_rsid.tsv.gz",
    snpid="SNPID",
    chrom="CHR",
    pos="POS",
    ea="EA",            # In your data, it's now EA (was ALT)
    nea="NEA",          # In your data, it's now NEA (was REF)
    eaf="EAF",          # In your data, it's now EAF (was minor_AF/Frq)
    beta="BETA",
    se="SE",
    p="P",
    # Note: 'Dir' and 'N' are not in your column list, 
    # so we should omit them or point to the correct ones.
    # n="n_complete_samples", # Optional: if you want to use sample size
    verbose=True,
    readargs={'sep': '\t'}
)

2026/01/14 08:09:08 GWASLab v4.0.4 https://cloufield.github.io/gwaslab/
2026/01/14 08:09:08 (C) 2022-2026, Yunye He, Kamatani Lab, GPL-3.0 license, gwaslab@gmail.com
2026/01/14 08:09:08 Python version: 3.12.3 (main, Jan  8 2026, 11:30:50) [GCC 13.3.0]
2026/01/14 08:09:08 Start to initialize gl.Sumstats from file :/mnt/hdd_1/ofgeha/galaxy-gwas-tools/Data/21001_T2T_liftover_rsid.tsv.gz
2026/01/14 08:09:33  -Reading columns          : BETA,P,CHR,POS,SNPID,EA,NEA,EAF,SE
2026/01/14 08:09:33  -Renaming columns to      : BETA,P,CHR,POS,SNPID,EA,NEA,EAF,SE
2026/01/14 08:09:33  -Current Dataframe shape : 13779885  x  9
2026/01/14 08:09:34  -Initiating a status column: STATUS ...
2026/01/14 08:09:34 #WARNING! Version of genomic coordinates is unknown for homo sapiens (build: 99)...
2026/01/14 08:09:37 Start to reorder the columns ...(v4.0.4)
2026/01/14 08:09:37  -Reordering columns to    : SNPID,CHR,POS,EA,NEA,STATUS,EAF,BETA,SE,P
2026/01/14 08:09:37 Finished reordering the columns.
2026/01/14 0

In [4]:
mysumstats.data.columns.tolist()

['SNPID', 'CHR', 'POS', 'EA', 'NEA', 'STATUS', 'EAF', 'BETA', 'SE', 'P']

In [5]:
# import gwaslab as gl

# GWASLab has a built-in helper to fetch some of these
# This will download the auto-chromosome dbSNP reference for hg38 which includes frequencies
gl.download_ref("1kg_dbsnp151_hg38_auto")

2026/01/14 08:11:19 Start to download  1kg_dbsnp151_hg38_auto  ...
2026/01/14 08:11:19  -Downloading to: /home/ofgeha/.gwaslab/1kg_dbsnp151_hg38_auto.txt.gz
2026/01/14 08:11:19  -File /home/ofgeha/.gwaslab/1kg_dbsnp151_hg38_auto.txt.gz exists.
2026/01/14 08:11:19  - Updating record in config file...
2026/01/14 08:11:19 Downloaded  1kg_dbsnp151_hg38_auto  successfully!


In [ ]:
# 1. Path to your local hg38 FASTA
ref_fasta = "/mnt/hdd_1/ofgeha/galaxy-gwas-tools/Data/hg38.fa"

# 2. Path to your local hg38 VCFs (using the {chrom} wildcard)
# GWASLab will replace {chrom} with 1, 2, 3... as it iterates
ref_vcf = "/mnt/hdd_1/ofgeha/gwas-sumstats-harmoniser/gwas-ref/homo_sapiens-chr{chrom}.vcf.gz"

# 3. Perform Harmonization
mysumstats.harmonize(
    ref_seq=ref_fasta,      # Checks alleles against the hg38 genome sequence
    ref_infer=ref_vcf,      # Checks strand and frequency against the VCFs
    ref_alt_freq="AF",      # Standard dbSNP frequency column name
    threads=6,              # Optimized for your 6 threads
    verbose=False,          # Keeps the console clean
    # log=False               # Prevents giant log files for 13M rows
)

2026/01/14 08:13:52 Start to check if NEA is aligned with reference sequence ...(v4.0.4)
2026/01/14 08:13:52  -Reference genome FASTA file: /mnt/hdd_1/ofgeha/galaxy-gwas-tools/Data/hg38.fa


In [6]:
# 1. Path to your local FASTA (Reference Sequence)
ref_fasta = "/mnt/hdd_1/ofgeha/galaxy-gwas-tools/Data/hg38.fa"

# 2. Path to your local Reference (using the {chrom} wildcard)
# We point to the .parquet files because they are much faster for frequency lookups
ref_vcf = "/mnt/hdd_1/ofgeha/gwas-sumstats-harmoniser/gwas-ref/homo_sapiens-chr{chrom}.parquet"

# 3. Perform Harmonization
mysumstats.harmonize(
    ref_seq=ref_fasta,      # Used to check if alleles match the human genome
    ref_infer=ref_vcf,      # Used to check AF (Allele Frequency) and strand alignment
    ref_alt_freq="AF",      # Note: Check your parquet files, it's usually just "AF" or "EAF"
    threads=6,              # Using your 6 threads
    # build="38" is assumed since you lifted over
)

2026/01/14 08:11:31 Start to check SNPID/rsID ...(v4.0.4)
2026/01/14 08:11:31  -Current Dataframe shape : 13779885 x 10 ; Memory usage: 963.05 MB
2026/01/14 08:11:31  -Checking SNPID data type...
2026/01/14 08:11:31  -Converted datatype for SNPID: object -> string
2026/01/14 08:11:32  -Checking if SNPID contains NA strings :na,NA,Na,Nan,NaN,<NA>,null,NULL,#N/A,#VALUE!,N/A,n/a,missing,...
2026/01/14 08:11:33  -Checking if SNPID is CHR:POS:NEA:EA...(separator: - ,: , _)


KeyboardInterrupt: 

In [ ]:
# Path to the FASTA you just successfully downloaded
ref_fasta = "/mnt/hdd_1/ofgeha/galaxy-gwas-tools/Data/hg38.fa"

# Path to the VCF (Update filename if you used gl.download_ref)
ref_vcf = "/home/ofgeha/.gwaslab/1kg_dbsnp151_hg38_auto.txt.gz"

mysumstats.harmonize(
    ref_seq=ref_fasta,
    ref_infer=ref_vcf,
    ref_alt_freq="EUR_AF", 
    basic_check=True,
    threads=4
)

In [ ]:
mysumstats.data

### Check summary

In [ ]:
mysumstats.summary()

In [ ]:
# Use ref_rsid_tsv because the file is a text table, not a VCF
mysumstats.assign_rsid(
    ref_rsid_tsv="/home/ofgeha/.gwaslab/1kg_dbsnp151_hg38_auto.txt.gz",
    threads=4
)

In [ ]:
# A. Manually fix Chromosomes to avoid the "target" error
mysumstats.data['CHR'] = mysumstats.data['CHR'].astype(str).str.replace('chr', '', case=False)
mysumstats.data['CHR'] = mysumstats.data['CHR'].replace({'X': '23', 'Y': '24', 'MT': '25'})

# B. Set the build globally for this object
mysumstats.set_build("38")

# C. Run harmonization (removing the 'build' keyword that caused the error)
mysumstats.harmonize(
    ref_seq="/mnt/hdd_1/ofgeha/gwas-sumstats-harmoniser/gwas-ref/hg38.fa",
    basic_check=False,
    threads=4
)